In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import to_date, col, explode
from pyspark.sql.types import *

In [ ]:
import os, pathlib, shutil
os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["PATH"]       = r"C:\hadoop\bin;" + os.environ["PATH"]
os.environ["AWS_PROFILE"] = "ryanj"
os.environ["AWS_REGION"]  = "us-west-2"
os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["PATH"] = rf'{os.environ["HADOOP_HOME"]}\bin;' + os.environ["PATH"]
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.hadoop:hadoop-aws:3.3.6 "
    "--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem "
    "--conf spark.hadoop.fs.s3a.aws.region=us-west-2 "
    "--conf spark.hadoop.fs.s3a.aws.credentials.provider=com.amazonaws.auth.DefaultAWSCredentialsProviderChain "
    "--conf spark.hadoop.io.native.lib.available=false "
    "--conf spark.hadoop.fs.s3a.fast.upload=true "
    "--conf spark.hadoop.fs.s3a.fast.upload.buffer=bytebuffer "
    "pyspark-shell"
)

In [3]:
spark = (SparkSession.builder
         .appName("sumo-s3a")
         # make sure S3A is wired up; you can keep these if they helped earlier
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.profile.ProfileCredentialsProvider")
         .config("spark.hadoop.fs.s3a.aws.region", "us-west-2")
         .getOrCreate())

C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
hc = spark.sparkContext._jsc.hadoopConfiguration()
print("S3A impl:", hc.get("fs.s3a.impl")) 

S3A impl: org.apache.hadoop.fs.s3a.S3AFileSystem


In [5]:
# make a writable local temp dir
spark_local = r"C:\tmp\spark"
pathlib.Path(spark_local).mkdir(parents=True, exist_ok=True)

In [6]:
print("Spark:", spark.version)
print("Hadoop:", spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())

Spark: 4.0.1
Hadoop: 3.4.1


In [7]:
spark.conf.set("spark.hadoop.fs.s3a.connection.establish.timeout", "30000")   # 30s
spark.conf.set("spark.hadoop.fs.s3a.threads.keepalivetime",        "60000")   # 60s
spark.conf.set("spark.hadoop.fs.s3a.connection.ttl",              "300000")   # 5m
spark.conf.set("spark.hadoop.fs.s3a.retry.interval",                 "500")   # 500ms
spark.conf.set("spark.hadoop.fs.s3a.retry.throttle.interval",        "100")   # 100ms

In [8]:
# convert human-friendly durations to plain integers (milliseconds)
hconf = spark.sparkContext._jsc.hadoopConfiguration()

fixes = {
    "fs.s3a.connection.establish.timeout": "30000",    # 30s
    "fs.s3a.threads.keepalivetime": "60000",           # 60s
    "fs.s3a.retry.interval": "500",                    # 500ms
    "fs.s3a.retry.throttle.interval": "100",           # 100ms
    "fs.s3a.connection.ttl": "300000",                 # 5m
    "fs.s3a.assumed.role.session.duration": "1800000"  # 30m
}
for k, v in fixes.items():
    hconf.set(k, v)

# (optional) double-check one
print("keepalive:", hconf.get("fs.s3a.threads.keepalivetime"))


keepalive: 60000


In [9]:
import re

hconf = spark.sparkContext._jsc.hadoopConfiguration()

# report any time-looking values with units
offenders = []
it = hconf.iterator()
pairs = []
while it.hasNext():
    e = it.next()
    pairs.append((e.getKey(), e.getValue()))

r_time = re.compile(r"^\s*\d+\s*(ms|s|m|h|d)\s*$", re.I)

for k, v in pairs:
    if r_time.match(str(v)):
        offenders.append((k, v))

print("Offenders before fix:")
for k, v in offenders:
    print(f"  {k} = {v}")

# convert unit-suffixed durations to milliseconds
UNIT_MS = {"ms":1, "s":1000, "m":60_000, "h":3_600_000, "d":86_400_000}

def to_ms_str(val):
    m = re.fullmatch(r"\s*(\d+)\s*(ms|s|m|h|d)\s*", str(val).lower())
    if not m: 
        return None
    n = int(m.group(1)); u = m.group(2)
    return str(n * UNIT_MS[u])

changed = []
for k, v in pairs:
    ms = to_ms_str(v)
    if ms and ms != v:
        hconf.set(k, ms)
        changed.append((k, v, ms))

print("\nNormalized to milliseconds:")
for k, old, new in changed:
    print(f"  {k}: {old} -> {new}")

# show any stragglers still using units
left = []
it = hconf.iterator()
while it.hasNext():
    e = it.next()
    if r_time.match(str(e.getValue())):
        left.append((e.getKey(), e.getValue()))
print("\nRemaining with units (should be empty):", left)


Offenders before fix:
  fs.s3a.vectored.read.max.merged.size = 2M
  yarn.router.subcluster.cleaner.interval.time = 60s
  yarn.resourcemanager.delegation.token.remove-scan-interval = 1h
  yarn.router.interceptor.user-thread-pool.keep-alive-time = 30s
  fs.s3a.multipart.purge.age = 24h
  yarn.federation.state-store.heartbeat.initial-delay = 30s
  hadoop.security.groups.shell.command.timeout = 0s
  hadoop.service.shutdown.timeout = 30s
  yarn.federation.state-store.clean-up-retry-sleep-time = 1s
  fs.s3a.multipart.threshold = 128M
  yarn.router.subcluster.heartbeat.expiration.time = 30m
  yarn.resourcemanager.delegation-token-renewer.thread-retry-interval = 60s
  fs.s3a.block.size = 32M
  yarn.router.submit.interval.time = 10ms
  yarn.federation.gpg.webapp.connect-timeout = 30s
  yarn.federation.state-store.sql.max-life-time = 30m
  fs.s3a.multipart.size = 64M
  yarn.federation.state-store.sql.conn-time-out = 10s
  fs.azure.sas.expiry.period = 90d
  yarn.resourcemanager.delegation-token-r

# bronze reads and silver writes

In [10]:
spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")  # avoids choking on stray files

df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/rikishi_matches/"))


In [11]:

# After your JSON read
df.printSchema()
# You'll likely see: root
#  |-- 0: array (nullable = true)
#  |    |-- element: struct (contains your fields)

rows = df.select(explode(col("records")).alias("match"))
flat = rows.select("match.*")   # expand struct into columns
flat.show(20, truncate=False)
flat.printSchema()

root
 |-- limit: long (nullable = true)
 |-- records: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bashoId: string (nullable = true)
 |    |    |-- day: long (nullable = true)
 |    |    |-- division: string (nullable = true)
 |    |    |-- eastId: long (nullable = true)
 |    |    |-- eastRank: string (nullable = true)
 |    |    |-- eastShikona: string (nullable = true)
 |    |    |-- kimarite: string (nullable = true)
 |    |    |-- matchNo: long (nullable = true)
 |    |    |-- westId: long (nullable = true)
 |    |    |-- westRank: string (nullable = true)
 |    |    |-- westShikona: string (nullable = true)
 |    |    |-- winnerEn: string (nullable = true)
 |    |    |-- winnerId: long (nullable = true)
 |    |    |-- winnerJp: string (nullable = true)
 |-- skip: long (nullable = true)
 |-- total: long (nullable = true)

+-------+---+--------+------+------------------+-----------+-----------+-------+------+------------------+-----------+-

convert to parquet

In [12]:
silver = (flat
          .dropDuplicates())          # typical dedupe key

In [13]:
silver = (silver
         .withColumn("year", F.col("bashoId").cast("string").substr(1,4))
          .withColumn("month", F.col("bashoId").cast("string").substr(5,2))
          .withColumn("match_date",
                     F.to_date(
                         F.concat_ws("-", F.col("year"), F.col("month"), F.lpad(F.col("day"), 2, "0")),
                             "yyyy-MM-dd"
                         )
                     )
          .withColumn(
              "match_id",
              F.concat_ws("", F.date_format(F.col("match_date"), "yyyyMMdd"), F.col("matchNo").cast("string")).cast("long")
          )
          .drop("year", "month")
         )

In [14]:
silver.show(10)

+-------+---+--------+------+------------------+-----------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+
|bashoId|day|division|eastId|          eastRank|eastShikona|   kimarite|matchNo|westId|          westRank|westShikona|   winnerEn|winnerId|winnerJp|match_date|  match_id|
+-------+---+--------+------+------------------+-----------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+
| 201501|  1|Makuuchi|  3835| Maegashira 7 East| Kyokutenho|   yorikiri|     10|   670| Maegashira 7 West| Chiyootori| Kyokutenho|    3835|        |2015-01-01|2015010110|
| 201407| 15|Makuuchi|  3835|Maegashira 12 East| Kyokutenho|   yorikiri|      2|  4126|Maegashira 15 West|  Tokitenku|  Tokitenku|    4126|        |2014-07-15| 201407152|
| 201309| 10|Makuuchi|  4177|Maegashira 13 West|    Homasho|   yorikiri|     10|  3835| Maegashira 6 West| Kyokutenho|    Homasho|    4177|      

In [15]:
# optional sanity checks:
hc = spark.sparkContext._jsc.hadoopConfiguration()
print("io.native.lib.available =", hc.get("io.native.lib.available"))
print("fs.s3a.fast.upload =", hc.get("fs.s3a.fast.upload"))
print("fs.s3a.fast.upload.buffer =", hc.get("fs.s3a.fast.upload.buffer"))

io.native.lib.available = false
fs.s3a.fast.upload = true
fs.s3a.fast.upload.buffer = bytebuffer


In [16]:
(silver.write
 .mode("overwrite")
 .option("compression", "snappy")
 .partitionBy("bashoId")                        # or by year/month/day
 .parquet("s3a://ryans-sumo-bucket/silver/rikishi_matches/"))

In [ ]:
df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/rikishis/"))

In [ ]:
rows = df.select(explode(col("records")).alias("rikishi"))
flat = rows.select("rikishi.*")   # expand struct into columns
silver = (flat
          .dropDuplicates())  
(silver.write
 .mode("overwrite")
 .option("compression", "snappy")
 .parquet("s3a://ryans-sumo-bucket/silver/rikishis/"))

In [ ]:
df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/rikishi_stats/"))

df = df.withColumn(
    "rikishi_id",
    F.regexp_extract(F.input_file_name(), r"rikishi_(\d+)\.json", 1)
)

rows = df.select(explode(col("records")).alias("rikishi_stats"))
flat = rows.select("rikishi.*")   # expand struct into columns



silver = (flat
          .dropDuplicates())  
(silver.write
 .mode("overwrite")
 .option("compression", "snappy")
 .parquet("s3a://ryans-sumo-bucket/silver/rikishi_stats/"))

In [ ]:
df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.json")         # only JSON files
      .json("s3a://ryans-sumo-bucket/sumo-api-calls/basho/"))
rows = df.select(explode(col("records")).alias("basho"))
flat = rows.select("basho.*")   # expand struct into columns
silver = (flat
          .dropDuplicates())  
(silver.write
 .mode("overwrite")
 .option("compression", "snappy")
 .partitionBy("location")
 .parquet("s3a://ryans-sumo-bucket/silver/bashos/"))

# silvers have been written

In [17]:
df = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.parquet")         # only JSON files
      .parquet("s3a://ryans-sumo-bucket/silver/rikishi_matches/"))

In [25]:
df.show(10)

+---+---------+------+------------------+--------------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+-------+
|day| division|eastId|          eastRank|   eastShikona|   kimarite|matchNo|westId|          westRank|westShikona|   winnerEn|winnerId|winnerJp|match_date|  match_id|westWin|
+---+---------+------+------------------+--------------+-----------+-------+------+------------------+-----------+-----------+--------+--------+----------+----------+-------+
|  8| Makuuchi|   794| Maegashira 3 East|         Terao|   yorikiri|     14|  4071|   Sekiwake 1 West|       Kaio|       Kaio|    4071|        |1996-05-08|1996050814|      1|
|  4|Makushita|  3827| Makushita 17 East|   Chiyotenzan|shitatenage|     24|  3837| Makushita 15 East| Kitazakura|Chiyotenzan|    3827|        |1996-05-04|1996050424|      0|
|  3|    Juryo|  3835|      Juryo 7 West|    Kyokutenho|   yorikiri|     12|  5078|      Juryo 1 West|Tokitsunada| Kyokutenho

In [21]:
df.count()

738480

In [26]:
matches = (df
      .withColumn(
        "westWin",
        F.when(F.col("winnerId") == F.col("westId"), F.lit(1)).otherwise(F.lit(0))
        )
     )

In [ ]:
rikishis = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.parquet")         # only JSON files
      .parquet("s3a://ryans-sumo-bucket/silver/rikishis/"))

rikishi_stats = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.parquet")         # only JSON files
      .parquet("s3a://ryans-sumo-bucket/silver/rikishi_stats/"))


rikishis = (spark.read
      .option("multiLine", "true")                # set to "false" if you have JSON Lines
      .option("mode", "PERMISSIVE")               # or "FAILFAST"
      .option("recursiveFileLookup", "true")
      .option("pathGlobFilter", "*.parquet")         # only JSON files
      .parquet("s3a://ryans-sumo-bucket/silver/bashos/"))